In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import validate, get_hierarchical_affinity, get_topic_specialization, print_topic_sample
from configure import get_config

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train_tmp, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

In [6]:
if len(instances_train_tmp) > config.size:
    instances_train = np.random.choice(instances_train_tmp, config.size, replace=False)
else:
    instances_train = instances_train_tmp

In [7]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [8]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [9]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [10]:
if 'sess' in globals(): sess.close()
model = HierarchicalNeuralTopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [ ]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    time_start = time.time()
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

    if epoch % 10 == 0:
        time_log = float(time.time() - time_start)
        
        # validate
        loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
        ppl_train = np.exp(np.mean(ppls_train))
        loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

        # test
        if ppl_dev < ppl_min:
            ppl_min = ppl_dev
            loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
            saver.save(sess, config.path_model, global_step=global_step_log)
            cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
            update_checkpoint(config, checkpoint, global_step_log)

        # visualize topic
        topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
        topics_freq_idxs = bow_idxs[topics_freq_indices]
        topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
        topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
        descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
        recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}

        depth_specs = get_topic_specialization(sess, model, instances_test)
        hierarchical_affinities = get_hierarchical_affinity(sess, model)

        # log
        clear_output()
        log_series = pd.Series([time_log, epoch, ct, \
                '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                '%.2f'%loss_dev, ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                '%.2f'%loss_test, ppl_test, \
                '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                index=log_df.columns)
        log_df.loc[global_step_log] = log_series
        display(log_df)
        cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
        print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

        # update tree
        if not config.static:
            config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
            if update_tree_flg:
                print(config.tree_idxs)
                name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                if 'sess' in globals(): sess.close()
                model = HierarchicalNeuralTopicModel(config)
                sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                saver = tf.train.Saver(max_to_keep=1)

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                           VALID:  \
             Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS   
499     10.823985    0  498  113.17  582  112.80  0.35  0.01  105.62   
5489     6.017365   10  498  111.48  629  110.53  0.95  0.00  103.84   
10479    4.722492   20  498  111.09  704  109.90  1.19  0.00  103.67   
15469    5.928862   30  498  110.87  752  109.52  1.35  0.00  103.46   
20459    4.811092   40  498  110.70  771  109.23  1.47  0.00  103.37   
25449    5.065201   50  498  110.58  780  109.02  1.56  0.00  103.26   
30439    4.812073   60  498  110.49  781  108.85  1.63  0.00  103.22   
35429    6.166653   70  498  110.41  782  108.71  1.70  0.00  103.20   
40419    5.062011   80  498  110.34  782  108.58  1.75  0.00  103.14   
45409    5.312927   90  498  110.28  782  108.47  1.80  0.00  103.14   
50399    5.174116  100  498  110.23  782  108.38  1.84  0.00  103.15   
55389    5.806659  110  498  110.18  782  108.31  1.87  0.00  103.09   
60379    6.397255  120  498  110.14  781  108.24  1.90  0.00  103.08   
65369    5.662284  130  498  110.11  780  108.18  1.93  0.00  103.12   
70359    6.937842  140  498  110.08  779  108.12  1.95  0.00  103.07   
75349    5.370413  150  498  110.05  778  108.07  1.98  0.00  103.11   
80339    7.003517  160  498  110.03  777  108.03  1.99  0.00  103.05   
85329    5.257078  170  498  110.00  776  107.99  2.01  0.00  103.04   
90319    5.225343  180  498  109.98  775  107.96  2.03  0.00  103.07   
95309    5.391799  190  498  109.97  774  107.92  2.04  0.00  103.03   
100299   6.832526  200  498  109.95  773  107.89  2.06  0.00  103.04   

                                         TEST:             SPEC:              \
               PPL     NLL    KL   REG    LOSS         PPL     1     2     3   
499     537.996399  105.34  0.27  0.00  104.09  545.321960  0.17  0.19  0.26   
5489    491.958008  102.68  1.16  0.00  102.37  495.870392  0.25  0.54  0.48   
10479   488.571747  102.22  1.45  0.00  102.22  490.935120  0.28  0.51  0.54   
15469   481.501312  101.84  1.62  0.00  102.01  484.543884  0.29  0.60  0.48   
20459   479.189545  101.61  1.76  0.00  101.87  481.015289  0.30  0.54  0.52   
25449   475.050751  101.45  1.81  0.00  101.74  476.694275  0.30  0.60  0.49   
30439   474.704742  101.36  1.87  0.00  101.67  475.402222  0.31  0.55  0.51   
35429   473.477661  101.25  1.96  0.00  101.60  473.185394  0.30  0.58  0.50   
40419   471.716095  101.14  2.00  0.00  101.54  471.914093  0.30  0.58  0.51   
45409   471.982483  101.13  2.02  0.00  101.54  471.914093  0.30  0.58  0.51   
50399   472.285980  101.11  2.03  0.00  101.54  471.914093  0.30  0.58  0.52   
55389   470.635864  101.03  2.06  0.00  101.46  469.929047  0.30  0.57  0.52   
60379   470.421814  101.01  2.07  0.00  101.43  469.123474  0.30  0.57  0.52   
65369   471.830841  101.02  2.09  0.00  101.43  469.123474  0.31  0.56  0.52   
70359   469.764587  100.97  2.10  0.00  101.44  468.849762  0.30  0.57  0.52   
75349   471.106476  101.00  2.11  0.00  101.44  468.849762  0.30  0.57  0.52   
80339   469.270264  100.93  2.12  0.00  101.41  468.074860  0.30  0.57  0.52   
85329   468.915955  100.91  2.13  0.00  101.40  467.869781  0.30  0.56  0.52   
90319   470.080078  100.93  2.14  0.00  101.40  467.869781  0.30  0.56  0.52   
95309   468.643250  100.89  2.14  0.00  101.40  467.744659  0.30  0.57  0.52   
100299  468.557220  100.88  2.15  0.00  101.39  467.225037  0.30  0.56  0.52   

       HIER:        
       CHILD OTHER  
499     0.85  0.77  
5489    0.29  0.19  
10479   0.25  0.14  
15469   0.24  0.12  
20459   0.31  0.10  
25449   0.27  0.10  
30439   0.30  0.10  
35429   0.31  0.10  
40419   0.31  0.10  
45409   0.31  0.10  
50399   0.32  0.09  
55389   0.31  0.10  
60379   0.31  0.10  
65369   0.31  0.11  
70359   0.31  0.10  
75349   0.31  0.11  
80339   0.31  0.10  
85329   0.31  0.10  
90319   0.31  0.11  
95309   0.32  0.10  
100299  0.31  0.11

0 R: 1.000 P: 0.378 quality back nice strap bought carry made lot - work
   1 R: 0.274 P: 0.111 sleeve protection air pro protect inside nice snug soft bit
     13 R: 0.032 P: 0.032 ; & pro perfectly air love nice feel color mac
     12 R: 0.131 P: 0.131 cover bottom keyboard color top hard screen mac nice plastic
   2 R: 0.169 P: 0.060 price $ smell amazon reviews inch bought item buy !
     23 R: 0.070 P: 0.070 ! color love perfectly picture blue pink purple recommend perfect
     21 R: 0.040 P: 0.040 return received item ordered seller shipping arrived time disappointed company
   5 R: 0.098 P: 0.065 pocket mouse small power room drive netbook cards cord carry
     51 R: 0.033 P: 0.033 ipad ! charger love perfect room perfectly mouse inside extra
   4 R: 0.081 P: 0.041 pockets pocket compartment room small plenty camera accessories carry items
     41 R: 0.040 P: 0.040 ! carry love room perfect ipad pockets plenty lots books
{0: [1, 2, 5, 3], 1: [12, 11], 2: [23, 22], 5: [52], 3: [3